# 🔌 Logic Gates Detection with YOLOv8 – Type & Rotation Classification

**Author:** Shahzaib Ahmad  
**Platform:** Kaggle (GPU-accelerated training)  
**Dataset:** [Logic Gates Type/Rotation Detection Dataset](https://www.kaggle.com/datasets/)

---

## 📋 Problem Statement

In digital circuit design and educational tools, **automatic recognition of logic gates** from circuit diagrams is a valuable capability. This project tackles a **multi-class object detection problem** where the goal is to:

1. **Detect** logic gates (AND, OR, NOT, NAND, NOR, XOR, XNOR) in circuit images
2. **Classify** each gate's **type** and **rotation orientation**

This is particularly useful for:
- 🎓 **Educational software** that can grade student circuit drawings
- 🔧 **Circuit digitization** tools converting hand-drawn schematics to digital formats
- 🤖 **Automation** in electronics manufacturing quality control

---

## 🧠 Approach & Model Selection

### Why Kaggle?

This notebook was built on **Kaggle Notebooks** because it makes ML experiments easy to run, reproduce, and share:

| Benefit | Why it matters |
|---------|----------------|
| **Free GPU acceleration** | Faster YOLO training without local hardware setup |
| **Reproducible environment** | Consistent Docker-backed runtime (fewer “works on my machine” issues) |
| **Dataset integration** | One-click access to datasets under `/kaggle/input` |
| **Versioned outputs** | Models, logs, and plots saved under `/kaggle/working` and downloadable |
| **Shareable notebook link** | Recruiters can open and review your full workflow end-to-end |

Kaggle’s file layout also influences the implementation:
- Read-only inputs live in `/kaggle/input/...`
- Writable artifacts (patched YAML, trained weights, ZIP outputs) go to `/kaggle/working/...`

### Model Configuration Choices

- **Model Size:** `yolov8n` (nano) – Fastest variant, ideal for this relatively simple detection task
- **Image Size:** 1024px – Higher resolution preserves gate details and orientation cues
- **Optimizer:** AdamW with cosine learning rate schedule for smooth convergence
- **Augmentation Strategy:** Carefully disabled rotation/flip augmentations since **rotation IS a class feature** we're detecting

---

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 📦 Step 1: Environment Setup

First, we set up the Kaggle environment and explore the available input files. Kaggle provides a pre-configured Python environment with essential data science libraries.

In [ ]:
!pip install -U ultralytics

## 🔧 Step 2: Install Ultralytics YOLOv8

We install the latest version of Ultralytics, which provides the YOLOv8 implementation. The `-U` flag ensures we get the most recent version with all bug fixes and improvements.

In [ ]:
import os, glob, zipfile
from pathlib import Path
import yaml

DATASET_ROOT = "/kaggle/input/logic-gates-typerotation-detection-dataset"

# Find data.yaml anywhere under the dataset root
candidates = glob.glob(os.path.join(DATASET_ROOT, "**", "data.yaml"), recursive=True)
if not candidates:
    raise FileNotFoundError(f"No data.yaml found under {DATASET_ROOT}. Check your dataset structure.")
found_yaml = candidates[0]
print("Found data.yaml:", found_yaml)

# Heuristic: choose the base directory that actually contains images/train and images/val
def find_base_dir(yaml_path, dataset_root):
    p = Path(yaml_path).parent
    # check current and a couple of ancestors
    for root in [p, p.parent, p.parent.parent]:
        if (root / "images" / "train").exists() and (root / "images" / "val").exists():
            return str(root)
    # fallback: search under the whole dataset root
    trains = glob.glob(os.path.join(dataset_root, "**", "images", "train"), recursive=True)
    for t in trains:
        root = Path(t).parent.parent  # .../<root>/images/train -> <root>
        if (root / "images" / "val").exists():
            return str(root)
    # last resort: parent of yaml
    return str(p)

base_dir = find_base_dir(found_yaml, DATASET_ROOT)
print("Using base path for YAML 'path:':", base_dir)

# Load, patch, save
with open(found_yaml, "r") as f:
    y = yaml.safe_load(f)

y["path"] = base_dir  # keep your existing 'train: images/train', 'val: images/val'
patched_yaml = "/kaggle/working/data_patched.yaml"
with open(patched_yaml, "w") as f:
    yaml.safe_dump(y, f, sort_keys=False)

print("Patched yaml saved to:", patched_yaml)
print("---- data_patched.yaml ----")
print(open(patched_yaml).read())


## 📂 Step 3: Dataset Configuration & YAML Patching

YOLO models require a `data.yaml` file that specifies:
- **Path** to the dataset root
- **Train/Val/Test** folder locations
- **Class names** and their indices

Since Kaggle datasets have a read-only structure, we need to:
1. **Locate** the original `data.yaml` in the input directory
2. **Patch** the `path` field to point to the correct base directory
3. **Save** a modified copy to our writable working directory

This ensures the model can find images and labels correctly during training.

In [ ]:
%%writefile /kaggle/working/train_kaggle.py
from ultralytics import YOLO
import os, torch, shutil, sys
from pathlib import Path

def main(data_yaml_path):
    # ---- config ----
    DATA_YAML = data_yaml_path
    MODEL = "yolov8n.pt"
    EPOCHS = 100
    IMGSZ = 1024
    LR0 = 0.003
    PATIENCE = 20
    PROJECT = Path("runs/train")
    RUN_NAME = "logic_gates_yolov8n"
    SEED = 42

    # ---- device setup ----
    USE_MULTI_GPU = True  # set to False to force single GPU with AutoBatch
    if torch.cuda.is_available():
        n = torch.cuda.device_count()
        if USE_MULTI_GPU and n >= 2:
            device = "0,1"
            batch = 16          # MUST be a multiple of GPU count (2). Adjust if OOM.
            workers = 8
            amp = True
            print(f"Using multi-GPU: {device}  | batch={batch}")
        else:
            device = "0"
            batch = -1          # AutoBatch OK on single GPU
            workers = 4
            amp = True
            print(f"Using single GPU: {device}  | batch=Auto (-1)")
    else:
        device = "cpu"
        batch = 8
        workers = 2
        amp = False
        os.environ["OMP_NUM_THREADS"] = "8"
        os.environ["MKL_NUM_THREADS"] = "8"
        try:
            torch.set_num_threads(8)
        except Exception:
            pass
        print("Using CPU")

    # ---- train ----
    model = YOLO(MODEL)
    model.train(
        data=DATA_YAML,
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=batch,
        lr0=LR0,
        patience=PATIENCE,
        project=str(PROJECT),
        name=RUN_NAME,
        pretrained=True,
        seed=SEED,
        device=device,
        workers=workers,
        amp=amp,
        optimizer="AdamW",
        cos_lr=True,

        # rotation-safe augs (rotation is part of your classes)
        hsv_h=0.0, hsv_s=0.0, hsv_v=0.0,
        degrees=0.0, shear=0.0,
        perspective=0.002,
        translate=0.04, scale=0.20,
        flipud=0.0, fliplr=0.0,
        mosaic=0.5, close_mosaic=30,
        mixup=0.0, copy_paste=0.0,

        plots=True
    )

    # ---- copy best weights to /kaggle/working for download ----
    best_src = PROJECT / RUN_NAME / "weights" / "best.pt"
    out_best = "/kaggle/working/best_model.pt"
    if best_src.exists():
        shutil.copy2(best_src, out_best)
        print(f"✅ Saved {out_best}")
    else:
        print("⚠️ best.pt not found at", best_src)

    # Optional scripted export (best_model.pt is the main file you need)
    try:
        model.export(format="pt")
        print("✅ Exported scripted .pt (optional)")
    except Exception as e:
        print("Export skipped:", e)

if __name__ == "__main__":
    if len(sys.argv) < 2:
        raise SystemExit("Usage: python train_kaggle.py /kaggle/working/data_patched.yaml")
    main(sys.argv[1])


## 🏋️ Step 4: Training Script Definition

Here we define the complete training pipeline as a standalone Python script. This approach has several advantages:

### Key Training Decisions:

| Parameter | Value | Rationale |
|-----------|-------|-----------|
| `epochs=100` | Extended training for convergence with early stopping |
| `imgsz=1024` | High resolution to capture fine gate details |
| `lr0=0.003` | Conservative learning rate for stable fine-tuning |
| `patience=20` | Early stopping to prevent overfitting |

### Critical Augmentation Choices:

Since **rotation is a class feature** (we're detecting gates at different orientations), we **disable spatial augmentations** that would confuse the model:

```python
degrees=0.0      # No random rotation
flipud=0.0       # No vertical flips  
fliplr=0.0       # No horizontal flips
shear=0.0        # No shearing
```

We keep **color augmentations disabled** (`hsv_h/s/v=0.0`) since logic gate detection relies on shape, not color.

### Multi-GPU Optimization:

The script automatically detects available GPUs and configures:
- **Dual GPU (Kaggle P100s):** Batch size 16, distributed training
- **Single GPU:** AutoBatch for optimal memory usage
- **CPU fallback:** Reduced batch size with multi-threading

In [ ]:
!python -u /kaggle/working/train_kaggle.py "/kaggle/working/data_patched.yaml"


## 🚀 Step 5: Execute Training

Now we run the training script! This will:
1. Load the pre-trained YOLOv8n weights (transfer learning from COCO)
2. Fine-tune on our logic gates dataset
3. Save the best model weights based on validation mAP
4. Generate training plots and metrics

**Expected runtime on Kaggle GPU:** ~30-60 minutes depending on dataset size.

In [ ]:
import os, zipfile

zip_path = "/kaggle/working/outputs.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as z:
    for fp in ["/kaggle/working/best_model.pt", "/kaggle/working/data_patched.yaml"]:
        if os.path.exists(fp):
            z.write(fp, arcname=os.path.basename(fp))
    # include training runs (plots, metrics)
    for root, _, files in os.walk("/kaggle/working/runs"):
        for fn in files:
            full = os.path.join(root, fn)
            rel = os.path.relpath(full, "/kaggle/working")
            z.write(full, arcname=rel)

print("Created:", zip_path, "\nDownload from the right-side Output panel.")


---

## 📊 Results & Key Takeaways

### What to Expect from Training:

After training completes, check the `runs/train/logic_gates_yolov8n/` folder for:

| File | Description |
|------|-------------|
| `results.png` | Training/validation loss curves |
| `confusion_matrix.png` | Per-class classification accuracy |
| `PR_curve.png` | Precision-Recall curve |
| `F1_curve.png` | F1 score vs confidence threshold |

### Model Performance Interpretation:

- **mAP@50** > 0.9: Excellent detection, ready for production
- **mAP@50** 0.7-0.9: Good performance, may need more data or tuning
- **mAP@50** < 0.7: Consider data quality, augmentation strategy, or larger model

### Next Steps:

1. **Inference:** Load `best_model.pt` with `YOLO()` for predictions
2. **Optimization:** Export to ONNX/TensorRT for faster inference
3. **Deployment:** Integrate into your circuit analysis application

---

## 🙏 Acknowledgments

- **Dataset:** Logic Gates Type/Rotation Detection Dataset on Kaggle
- **Framework:** [Ultralytics YOLOv8](https://github.com/ultralytics/ultralytics)
- **Platform:** Kaggle Notebooks with GPU acceleration

---

*This notebook was developed on Kaggle using their free GPU resources. For best results, run with GPU acceleration enabled.*

## 📤 Step 6: Package Outputs for Download

Finally, we bundle all training artifacts into a single ZIP file for easy download:

- **`best_model.pt`** – The trained model weights (use this for inference!)
- **`data_patched.yaml`** – Dataset configuration for reproducibility
- **`runs/`** – Training logs, metrics, and visualization plots